In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

fun_charge = 1.601e-19
amu = 1.66056e-27

def distances(num_RF, freq, V_max, V_gained, e_inject, mass, charge):
    #find energies after each RF gap
    energies = [e_inject+charge*V_gained*V_max*i for i in range(num_RF)]

    #find distances between RF gaps
    distances = []
    for energy in energies:
        distances.append(np.sqrt(energy*charge*fun_charge/(2*mass*amu))*1/freq)

    return [energies, distances]

def draw_latice(num_RF, freq, V_max, V_gained, e_inject, mass, charge, mass2, charge2, n):
    #draw the energy and RF gap position
    d_and_e = distances(num_RF, freq, V_max, V_gained, e_inject, mass, charge)

    t_offset = np.arcsin(V_gained)/(2*np.pi*freq*n)-d_and_e[1][0]/np.sqrt(2*e_inject*charge*fun_charge/(mass*amu))
    energies2 = species_2_energies(d_and_e[1], freq, V_max, e_inject, mass2, charge2, t_offset, n)
    energies2 = energies2[0:-1]

    total_distances = np.cumsum(d_and_e[1])
    total_distances = np.append([0],total_distances[0:-1])
    energies = d_and_e[0]

#     print('distances:',total_distances)
#     print('energies:',energies)
#     print('energies2:',energies2)

    plt.step(total_distances, energies, color='green', where='post')
    plt.step(total_distances, energies2, color='red', where='post')

    plt.show()

def species_2_energies(distances, freq, V_max, e_inject, mass, charge, t_offset, n):
    energies = [e_inject]
    energy = e_inject
    time = 0
    for d, i in zip(distances, range(len(distances))):
        velocity = np.sqrt(2*energy*fun_charge/(mass*amu))
        time = time + d/velocity
        energy = energy+(charge*V_max*np.sin(2*np.pi*freq*n*(time+t_offset)))*(-1)**(i)
        energies.append(energy)
    return energies

drawing = interact(draw_latice, num_RF=30, freq=20e6, V_max=1000, V_gained=.8, e_inject=20e3, mass=20, charge=1, mass2=20, charge2=1, n=1)


interactive(children=(IntSlider(value=30, description='num_RF', max=90, min=-30), FloatSlider(value=20000000.0…